# Stacks & Queues

*LIFO and FIFO abstract data types*


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⏱️ OTURUM ZAMANLAYICI — Bu hücreyi ilk çalıştırın!
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import time as _time, datetime as _dt, json as _json, os as _os
from IPython.display import display, HTML as _HTML

# ── Persistent tracker file (survives kernel restarts in same runtime) ──
_TRACKER_FILE = _os.path.join(
    '/content' if _os.path.isdir('/content') else '/tmp',
    '.cp2_session_Week_05.json'
)

def _load_tracker():
    """Load previous sessions from file."""
    try:
        with open(_TRACKER_FILE, 'r') as f:
            return _json.load(f)
    except (FileNotFoundError, _json.JSONDecodeError, ValueError):
        return {'sessions': [], 'total_heartbeats': 0}

def _save_tracker(data):
    """Persist tracker data to file."""
    try:
        with open(_TRACKER_FILE, 'w') as f:
            _json.dump(data, f)
    except OSError:
        pass

# ── Load any previous session data ──
_tracker = _load_tracker()
_prev_sessions = len(_tracker['sessions'])
_prev_hb = _tracker['total_heartbeats']

# ── Start new session ──
_SESSION_START = _time.time()
_SESSION_START_STR = _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
_HEARTBEATS = [_time.time()]
_CELLS_RUN = [0]
IDLE_THRESHOLD = 300   # 5 min

# Record this session start
_tracker['sessions'].append({
    'start': _SESSION_START_STR,
    'start_ts': _SESSION_START,
    'heartbeats': [_SESSION_START]
})
_save_tracker(_tracker)

def _heartbeat_hook(*args, **kwargs):
    """Record each cell execution — in-memory + file."""
    now = _time.time()
    _HEARTBEATS.append(now)
    _CELLS_RUN[0] += 1
    # Persist every 5 cells to avoid excessive disk IO
    if _CELLS_RUN[0] % 5 == 0:
        try:
            _t = _load_tracker()
            if _t['sessions']:
                _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
                _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
                _t['total_heartbeats'] = _prev_hb + len(_HEARTBEATS)
                _save_tracker(_t)
        except Exception:
            pass

def _calc_active_time(heartbeats=None):
    """Calculate active time from heartbeat list, capping idle gaps."""
    hb = heartbeats or _HEARTBEATS
    if len(hb) < 2: return 0
    active = 0
    for i in range(1, len(hb)):
        gap = hb[i] - hb[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _calc_total_active_time():
    """Calculate total across ALL sessions (multi-day support)."""
    total = 0
    try:
        _t = _load_tracker()
        for sess in _t['sessions'][:-1]:  # all previous sessions
            hb = sess.get('heartbeats', [])
            if len(hb) >= 2:
                for i in range(1, len(hb)):
                    gap = hb[i] - hb[i-1]
                    total += gap if gap <= IDLE_THRESHOLD else 30
    except Exception:
        pass
    # Add current session
    total += _calc_active_time()
    return int(total)

try:
    _ip = get_ipython()
    _ip.events.register('pre_run_cell', _heartbeat_hook)
except Exception: pass

# ── Display ──
_multi = ''
if _prev_sessions > 0:
    _multi = f'<br><span style="font-size:12px;opacity:.75">📂 {_prev_sessions} önceki oturum bulundu — çoklu oturum takibi aktif.</span>'
display(_HTML(f"""<div style='background:linear-gradient(135deg,#667eea,#764ba2);padding:14px 20px;border-radius:10px;color:white;font-family:system-ui;margin:4px 0'>
<b>⏱️ Oturum Başladı</b> — {_SESSION_START_STR}<br>
<span style='font-size:13px;opacity:.85'>Hücre aktiviteniz takip ediliyor. Bitince en alttaki Submit hücresini çalıştırın.</span>{_multi}</div>"""))
print(f'✅ Zamanlayıcı aktif. Idle eşiği: {IDLE_THRESHOLD//60} dk')
if _prev_sessions > 0:
    print(f'📂 Önceki oturumlar: {_prev_sessions} | Toplam heartbeat: {_prev_hb}')

## 🎯 Learning Objectives

- Understand the LIFO (stack) and FIFO (queue) abstract data types and their real-world analogies
- Implement stacks and queues using both arrays and linked lists, understanding trade-offs
- Apply stacks to solve parenthesis matching, expression evaluation, and next-greater-element problems
- Implement queue variants: circular queue, deque, priority queue, and monotonic queue
- Recognize when a problem requires stack-based or queue-based approaches

---
## 1. Introduction to Stacks and Queues


In the previous weeks, we explored arrays and linked lists—fundamental data structures that store collections of elements. Now we introduce **stacks** and **queues**, which are **abstract data types (ADTs)** that restrict how we access elements. These restrictions make them perfect for specific problems.

> 📖 **Definition:** An **Abstract Data Type (ADT)** defines what operations can be performed on data, not how they're implemented. Stacks and queues are ADTs that can be implemented using arrays or linked lists, but they enforce specific access patterns: LIFO (Last In, First Out) for stacks, and FIFO (First In, First Out) for queues.

### Real-World Analogies

> 💡 **Stack vs Queue in Daily Life**
>
> ```

STACK (LIFO - Last In, First Out):
┌─────────────────────────────────────────┐
│  • Stack of plates in cafeteria         │
│    → Take from top, add to top          │
│  • Undo history in text editor          │
│    → Last action is first to undo       │
│  • Browser back button                  │
│    → Last page visited, first to return │
│  • Function call stack                  │
│    → Last function called, first to end │
└─────────────────────────────────────────┘

QUEUE (FIFO - First In, First Out):
┌─────────────────────────────────────────┐
│  • Line at coffee shop                  │
│    → First in line, first served        │
│  • Print queue                          │
│    → First document sent, first printed │
│  • Task scheduling                      │
│    → Tasks processed in order received  │
│  • Breadth-first search                 │
│    → Explore nodes level by level       │
└─────────────────────────────────────────┘
                
```

**Listing 5.1 — Stack vs Queue Behavior**

In [ ]:
# Demonstrate the key difference between stacks and queues
from collections import deque

# Add elements 1, 2, 3 to both
stack = []  # Use list as stack
queue = deque()  # Use deque as queue

for x in [1, 2, 3]:
    stack.append(x)
    queue.append(x)

print("After adding 1, 2, 3:")
print(f"  Stack: {stack}")
print(f"  Queue: {list(queue)}")

# Remove elements
print("\nRemoving elements:")
print(f"  Stack removes: {stack.pop()}, {stack.pop()}, {stack.pop()}")  # 3, 2, 1

queue = deque([1, 2, 3])  # Reset queue
print(f"  Queue removes: {queue.popleft()}, {queue.popleft()}, {queue.popleft()}")  # 1, 2, 3

print("\nStack: Last In = First Out (LIFO)")
print("Queue: First In = First Out (FIFO)")

***Figure 5.1:** Same elements added, but removal order differs: stack removes most recent, queue removes oldest.*

### Why Restrict Access?

You might wonder: why limit ourselves when arrays give us full access? The restrictions of stacks and queues actually provide benefits:

- **Simplicity:** Fewer operations mean simpler code and fewer bugs
- **Efficiency:** All operations are O(1)
- **Correctness:** The structure enforces the right access pattern for your algorithm
- **Clarity:** Code intent is clear—a stack says "I need LIFO behavior"

---
## 2. The Stack: LIFO Principle


> 📖 **Definition:** A **stack** is a linear data structure that follows the **Last In, First Out (LIFO)** principle. Elements can only be added to (pushed) or removed from (popped) the top of the stack. Think of it as a stack of plates—you can only access the top plate.

### Stack Operations

| Operation | Description | Time |
| --- | --- | --- |
| `push(x)` | Add element x to top of stack | O(1) |
| `pop()` | Remove and return top element | O(1) |
| `peek()` / `top()` | Return top element without removing | O(1) |
| `is_empty()` | Check if stack has no elements | O(1) |
| `size()` | Return number of elements | O(1) |

> 💡 **Stack Operations Visualization**
>
> ```

Empty stack:     push(10):       push(20):       push(30):
    ┌───┐           ┌───┐           ┌───┐           ┌───┐
    │   │           │   │           │   │           │30 │ ← top
    ├───┤           ├───┤           ├───┤           ├───┤
    │   │           │   │           │20 │ ← top     │20 │
    ├───┤           ├───┤           ├───┤           ├───┤
    │   │           │10 │ ← top     │10 │           │10 │
    └───┘           └───┘           └───┘           └───┘

pop() returns 30:    pop() returns 20:    peek() returns 10:
    ┌───┐               ┌───┐               ┌───┐
    │   │               │   │               │   │
    ├───┤               ├───┤               ├───┤
    │20 │ ← top         │   │               │   │
    ├───┤               ├───┤               ├───┤
    │10 │               │10 │ ← top         │10 │ ← top (unchanged)
    └───┘               └───┘               └───┘
                
```

**Listing 5.2 — Basic Stack Operations**

In [ ]:
# Python list as a stack (most common approach)
stack = []

# push() - use append()
stack.append(10)
stack.append(20)
stack.append(30)
print(f"After pushing 10, 20, 30: {stack}")

# peek() - access last element
print(f"Peek (top element): {stack[-1]}")

# pop() - removes and returns
top = stack.pop()
print(f"Popped: {top}")
print(f"Stack after pop: {stack}")

# is_empty() - check length
print(f"Is empty: {len(stack) == 0}")

# size()
print(f"Size: {len(stack)}")

# Pop all remaining
while stack:
    print(f"Popping: {stack.pop()}")
    
print(f"Final stack: {stack}")

***Figure 5.2:** Python lists support efficient stack operations via `append()` and `pop()`.*

**Listing 5.3 — Stack Performance Comparison**

In [ ]:
import time

n = 5000

# Correct: append/pop at end - O(1)
stack = []
start = time.perf_counter()
for i in range(n):
    stack.append(i)
for i in range(n):
    stack.pop()
correct_time = time.perf_counter() - start

# Wrong: insert/pop at beginning - O(n)
stack = []
start = time.perf_counter()
for i in range(n):
    stack.insert(0, i)  # O(n) each time!
for i in range(n):
    stack.pop(0)        # O(n) each time!
wrong_time = time.perf_counter() - start

print(f"Stack operations ({n} push + {n} pop):")
print(f"  Correct (append/pop):    {correct_time*1000:.2f} ms")
print(f"  Wrong (insert(0)/pop(0)): {wrong_time*1000:.2f} ms")
speedup = wrong_time/correct_time if correct_time > 0 else float('inf')
print(f"  Speedup: {speedup:.0f}x")
print(f"\nAlways use the correct end of the list!")

***Figure 5.3:** Using the wrong end of a list makes operations O(n) instead of O(1).*

---
## 3. Stack Implementations


### Implementation 1: Array-Based Stack

**Listing 5.3 — Array-Based Stack Class**

In [ ]:
class ArrayStack:
    """Stack implementation using a Python list."""
    
    def __init__(self):
        self._data = []
    
    def __len__(self):
        return len(self._data)
    
    def __str__(self):
        return f"Stack({self._data})"
    
    def is_empty(self):
        return len(self._data) == 0
    
    def push(self, item):
        """Add item to top of stack - O(1) amortized."""
        self._data.append(item)
    
    def pop(self):
        """Remove and return top item - O(1)."""
        if self.is_empty():
            raise IndexError("Pop from empty stack")
        return self._data.pop()
    
    def peek(self):
        """Return top item without removing - O(1)."""
        if self.is_empty():
            raise IndexError("Peek at empty stack")
        return self._data[-1]

# Test
stack = ArrayStack()
for x in [1, 2, 3, 4, 5]:
    stack.push(x)
    
print(f"Stack: {stack}")
print(f"Size: {len(stack)}")
print(f"Peek: {stack.peek()}")
print(f"Pop: {stack.pop()}")
print(f"After pop: {stack}")

***Figure 5.3:** Array-based stack is simple and cache-friendly. Dynamic array handles resizing automatically.*

### Implementation 2: Linked List-Based Stack

**Listing 5.4 — Linked List-Based Stack**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedStack:
    """Stack implementation using a singly linked list."""
    
    def __init__(self):
        self._head = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def __str__(self):
        items = []
        current = self._head
        while current:
            items.append(str(current.data))
            current = current.next
        return f"Stack[{' -> '.join(items)}]"
    
    def is_empty(self):
        return self._head is None
    
    def push(self, item):
        """Add item to top (front of list) - O(1)."""
        new_node = Node(item)
        new_node.next = self._head
        self._head = new_node
        self._size += 1
    
    def pop(self):
        """Remove and return top item - O(1)."""
        if self.is_empty():
            raise IndexError("Pop from empty stack")
        data = self._head.data
        self._head = self._head.next
        self._size -= 1
        return data
    
    def peek(self):
        """Return top item without removing - O(1)."""
        if self.is_empty():
            raise IndexError("Peek at empty stack")
        return self._head.data

# Test
stack = LinkedStack()
for x in ['A', 'B', 'C', 'D']:
    stack.push(x)
    
print(f"Stack: {stack}")
print(f"Peek: {stack.peek()}")
print(f"Pop: {stack.pop()}")
print(f"After pop: {stack}")

***Figure 5.4:** Linked list stack uses the head as the top. Push/pop at head gives O(1) without amortization.*

### Comparison: Array vs Linked List Stack

| Aspect | Array-Based | Linked List-Based |
| --- | --- | --- |
| push() | O(1) amortized | O(1) worst case |
| pop() | O(1) | O(1) |
| Memory | Compact, cache-friendly | Extra pointer overhead |
| Max size | Can resize (or fixed) | Limited only by memory |
| Best for | Most general use | When worst-case O(1) required |

**Listing 5.5 — Fixed-Size Array Stack**

In [ ]:
class FixedStack:
    """Stack with maximum capacity (no resizing)."""
    
    def __init__(self, capacity):
        self._data = [None] * capacity
        self._capacity = capacity
        self._top = -1  # Index of top element (-1 = empty)
    
    def __len__(self):
        return self._top + 1
    
    def is_empty(self):
        return self._top == -1
    
    def is_full(self):
        return self._top == self._capacity - 1
    
    def push(self, item):
        if self.is_full():
            raise OverflowError("Stack is full")
        self._top += 1
        self._data[self._top] = item
    
    def pop(self):
        if self.is_empty():
            raise IndexError("Pop from empty stack")
        item = self._data[self._top]
        self._data[self._top] = None  # Help garbage collection
        self._top -= 1
        return item
    
    def peek(self):
        if self.is_empty():
            raise IndexError("Peek at empty stack")
        return self._data[self._top]

# Test with capacity 3
stack = FixedStack(3)
print(f"Capacity: 3, Empty: {stack.is_empty()}")

stack.push(1)
stack.push(2)
stack.push(3)
print(f"After pushing 1,2,3: Full={stack.is_full()}")

try:
    stack.push(4)
except OverflowError as e:
    print(f"Error on push(4): {e}")

print(f"Popped: {stack.pop()}")
print(f"Now full: {stack.is_full()}")

***Figure 5.5:** Fixed-size stack is useful when you know the maximum size and want guaranteed O(1) without resizing.*

---
## 4. Stack Applications


### Application 1: Balanced Parentheses

**Listing 5.6 — Balanced Parentheses Checker**

In [ ]:
def is_balanced(s):
    """
    Check if brackets are balanced.
    Supports (), [], {}
    """
    stack = []
    matching = {')': '(', ']': '[', '}': '{'}
    
    for char in s:
        if char in '([{':
            stack.append(char)
        elif char in ')]}':
            if not stack:
                return False  # Closing without opening
            if stack.pop() != matching[char]:
                return False  # Mismatched pair
    
    return len(stack) == 0  # Stack should be empty

# Test cases
test_cases = [
    "()",
    "()[]{}",
    "(]",
    "([)]",
    "{[()]}",
    "(((",
    "a + (b * [c - d])",
    "",
]

print("Balanced parentheses check:")
for s in test_cases:
    result = "✓" if is_balanced(s) else "✗"
    display = f"'{s}'" if s else "empty"
    print(f"  {result} {display}")

***Figure 5.6:** Classic stack application: push opening brackets, pop and match for closing brackets.*

### Application 2: Reverse Polish Notation (RPN) Calculator

**Listing 5.7 — RPN Calculator**

In [ ]:
def evaluate_rpn(tokens):
    """
    Evaluate Reverse Polish Notation expression.
    Example: ["2", "3", "+", "4", "*"] = (2 + 3) * 4 = 20
    """
    stack = []
    operators = {
        '+': lambda a, b: a + b,
        '-': lambda a, b: a - b,
        '*': lambda a, b: a * b,
        '/': lambda a, b: int(a / b),  # Integer division toward zero
    }
    
    for token in tokens:
        if token in operators:
            b = stack.pop()  # Second operand (popped first!)
            a = stack.pop()  # First operand
            result = operators[token](a, b)
            stack.append(result)
        else:
            stack.append(int(token))
    
    return stack[0]

# Test cases
expressions = [
    (["2", "3", "+"], "2 + 3"),
    (["2", "3", "+", "4", "*"], "(2 + 3) * 4"),
    (["4", "13", "5", "/", "+"], "4 + (13 / 5)"),
    (["10", "6", "9", "3", "+", "-11", "*", "/", "*", "17", "+", "5", "+"],
     "complex expression"),
]

print("RPN Calculator:")
for tokens, description in expressions:
    result = evaluate_rpn(tokens)
    print(f"  {' '.join(tokens)}")
    print(f"    = {description} = {result}\n")

***Figure 5.7:** RPN eliminates parentheses by placing operators after operands. Stack makes evaluation simple.*

### Application 3: Infix to Postfix Conversion

**Listing 5.8 — Shunting Yard Algorithm**

In [ ]:
def infix_to_postfix(expression):
    """
    Convert infix expression to postfix using Shunting Yard algorithm.
    Example: "3 + 4 * 2" -> "3 4 2 * +"
    """
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3}
    right_associative = {'^'}
    
    output = []
    operator_stack = []
    
    tokens = expression.split()
    
    for token in tokens:
        if token.isdigit():
            output.append(token)
        elif token in precedence:
            while (operator_stack and 
                   operator_stack[-1] != '(' and
                   operator_stack[-1] in precedence and
                   (precedence[operator_stack[-1]] > precedence[token] or
                    (precedence[operator_stack[-1]] == precedence[token] and
                     token not in right_associative))):
                output.append(operator_stack.pop())
            operator_stack.append(token)
        elif token == '(':
            operator_stack.append(token)
        elif token == ')':
            while operator_stack and operator_stack[-1] != '(':
                output.append(operator_stack.pop())
            operator_stack.pop()  # Remove '('
    
    while operator_stack:
        output.append(operator_stack.pop())
    
    return ' '.join(output)

# Test
expressions = [
    "3 + 4",
    "3 + 4 * 2",
    "( 3 + 4 ) * 2",
    "3 + 4 * 2 / ( 1 - 5 )",
]

print("Infix to Postfix conversion:")
for expr in expressions:
    postfix = infix_to_postfix(expr)
    print(f"  {expr}")
    print(f"    → {postfix}\n")

***Figure 5.10:** Dijkstra's Shunting Yard algorithm converts infix to postfix using operator precedence.*

### Application 4: Decode String

**Listing 5.11 — Decode Encoded String**

In [ ]:
def decode_string(s):
    """
    Decode string like "3[a2[c]]" -> "accaccacc"
    Uses stack to handle nested brackets.
    """
    stack = []
    current_string = ""
    current_num = 0
    
    for char in s:
        if char.isdigit():
            current_num = current_num * 10 + int(char)
        elif char == '[':
            # Save current state and start fresh
            stack.append((current_string, current_num))
            current_string = ""
            current_num = 0
        elif char == ']':
            # Pop and build repeated string
            prev_string, num = stack.pop()
            current_string = prev_string + current_string * num
        else:
            current_string += char
    
    return current_string

# Test cases
test_cases = [
    "3[a]",
    "3[a2[c]]",
    "2[abc]3[cd]ef",
    "10[a]",
]

print("Decode String:")
for s in test_cases:
    result = decode_string(s)
    print(f"  '{s}' -> '{result}'")

***Figure 5.11:** Nested encoding is handled by pushing state onto stack at each '['.*

### Application 5: Basic Calculator

**Listing 5.12 — Basic Calculator with +, -, (, )**

In [ ]:
def calculate(s):
    """
    Evaluate expression with +, -, (, ).
    Uses stack for parentheses handling.
    """
    stack = []
    result = 0
    num = 0
    sign = 1
    
    for char in s:
        if char.isdigit():
            num = num * 10 + int(char)
        elif char == '+':
            result += sign * num
            num = 0
            sign = 1
        elif char == '-':
            result += sign * num
            num = 0
            sign = -1
        elif char == '(':
            # Save current result and sign
            stack.append(result)
            stack.append(sign)
            result = 0
            sign = 1
        elif char == ')':
            result += sign * num
            num = 0
            # Pop sign and previous result
            result *= stack.pop()  # sign
            result += stack.pop()  # previous result
    
    result += sign * num
    return result

# Test cases
expressions = [
    "1 + 1",
    " 2-1 + 2 ",
    "(1+(4+5+2)-3)+(6+8)",
    "2-(5-6)",
]

print("Basic Calculator:")
for expr in expressions:
    result = calculate(expr)
    print(f"  {expr} = {result}")

***Figure 5.12:** Calculator uses stack to save state before each '(' and restore after ')'.*

### Application 6: Function Call Stack Simulation

**Listing 5.9 — Call Stack Visualization**

In [ ]:
# Simulate how the call stack works during recursion
call_stack = []

def factorial(n, depth=0):
    """Factorial with call stack visualization."""
    # Push current call onto stack
    call_stack.append(f"factorial({n})")
    print(f"{'  ' * depth}CALL: factorial({n})")
    print(f"{'  ' * depth}Stack: {call_stack}")
    
    if n <= 1:
        result = 1
    else:
        result = n * factorial(n - 1, depth + 1)
    
    # Pop current call from stack
    call_stack.pop()
    print(f"{'  ' * depth}RETURN: factorial({n}) = {result}")
    print(f"{'  ' * depth}Stack: {call_stack}")
    
    return result

print("Call stack during factorial(4):\n")
result = factorial(4)
print(f"\nFinal result: {result}")

***Figure 5.9:** Every function call pushes a frame onto the call stack; every return pops it.*

### Application 5: Undo/Redo Functionality

**Listing 5.10 — Undo/Redo with Two Stacks**

In [ ]:
class TextEditor:
    """Simple text editor with undo/redo using two stacks."""
    
    def __init__(self):
        self.text = ""
        self.undo_stack = []  # Stores previous states
        self.redo_stack = []  # Stores undone states
    
    def write(self, s):
        """Add text."""
        self.undo_stack.append(self.text)  # Save current state
        self.redo_stack.clear()  # Clear redo after new action
        self.text += s
        print(f"Write '{s}': '{self.text}'")
    
    def undo(self):
        """Undo last action."""
        if not self.undo_stack:
            print("Nothing to undo")
            return
        self.redo_stack.append(self.text)  # Save for redo
        self.text = self.undo_stack.pop()  # Restore previous
        print(f"Undo: '{self.text}'")
    
    def redo(self):
        """Redo last undone action."""
        if not self.redo_stack:
            print("Nothing to redo")
            return
        self.undo_stack.append(self.text)
        self.text = self.redo_stack.pop()
        print(f"Redo: '{self.text}'")

# Demo
editor = TextEditor()
editor.write("Hello")
editor.write(" World")
editor.write("!")
print()
editor.undo()
editor.undo()
print()
editor.redo()
print()
editor.write(" Python")  # This clears redo stack
editor.redo()  # Nothing to redo

***Figure 5.10:** Two stacks enable both undo and redo: undo pops from undo_stack to redo_stack, redo reverses.*

### Application 6: Next Greater Element

**Listing 5.11 — Monotonic Stack Pattern**

In [ ]:
def next_greater_element(arr):
    """
    For each element, find the next element that is greater.
    Return -1 if no greater element exists.
    Uses a monotonic decreasing stack.
    Time: O(n), Space: O(n)
    """
    n = len(arr)
    result = [-1] * n
    stack = []  # Stores indices of elements waiting for their NGE
    
    for i in range(n):
        # While current element is greater than stack top
        while stack and arr[i] > arr[stack[-1]]:
            idx = stack.pop()
            result[idx] = arr[i]
        stack.append(i)
    
    return result

def next_greater_circular(arr):
    """Next greater element in circular array."""
    n = len(arr)
    result = [-1] * n
    stack = []
    
    # Go around twice to handle circular nature
    for i in range(2 * n):
        while stack and arr[i % n] > arr[stack[-1]]:
            idx = stack.pop()
            result[idx] = arr[i % n]
        if i < n:
            stack.append(i)
    
    return result

# Test
arr = [4, 5, 2, 25, 7, 8]
print(f"Array: {arr}")
print(f"Next greater: {next_greater_element(arr)}")

arr2 = [1, 2, 1]
print(f"\nCircular array: {arr2}")
print(f"Next greater (circular): {next_greater_circular(arr2)}")

***Figure 5.11:** Monotonic stack finds next greater/smaller elements in O(n) time.*

---
## 5. The Queue: FIFO Principle


> 📖 **Definition:** A **queue** is a linear data structure that follows the **First In, First Out (FIFO)** principle. Elements are added at the **rear** (enqueue) and removed from the **front** (dequeue). Think of it as a line of people—first in line is first served.

### Queue Operations

| Operation | Description | Time |
| --- | --- | --- |
| `enqueue(x)` | Add element x to rear | O(1) |
| `dequeue()` | Remove and return front element | O(1) |
| `front()` / `peek()` | Return front element without removing | O(1) |
| `is_empty()` | Check if queue has no elements | O(1) |
| `size()` | Return number of elements | O(1) |

> 💡 **Queue Operations Visualization**
>
> ```

Empty queue:
    front → [           ] ← rear

enqueue(10):
    front → [ 10        ] ← rear

enqueue(20):
    front → [ 10 | 20   ] ← rear

enqueue(30):
    front → [ 10 | 20 | 30 ] ← rear

dequeue() returns 10:
    front → [ 20 | 30   ] ← rear

dequeue() returns 20:
    front → [ 30        ] ← rear
                
```

**Listing 5.12 — Basic Queue Operations**

In [ ]:
from collections import deque

# Python deque as a queue (recommended)
queue = deque()

# enqueue() - use append()
queue.append(10)
queue.append(20)
queue.append(30)
print(f"After enqueueing 10, 20, 30: {list(queue)}")

# peek() - access first element
print(f"Front element: {queue[0]}")

# dequeue() - use popleft()
front = queue.popleft()
print(f"Dequeued: {front}")
print(f"Queue after dequeue: {list(queue)}")

# is_empty() - check length
print(f"Is empty: {len(queue) == 0}")

# Dequeue all
print("\nDequeuing all:")
while queue:
    print(f"  Dequeued: {queue.popleft()}")
    
print(f"Final queue: {list(queue)}")

***Figure 5.16:** Python's `deque` provides O(1) operations at both ends—perfect for queues.*

**Listing 5.17 — Queue Interface Comparison**

In [ ]:
from collections import deque
import queue

# Three ways to use queues in Python

# 1. collections.deque (most common for general use)
q1 = deque()
q1.append(1)
q1.append(2)
print(f"deque: {list(q1)}, popleft: {q1.popleft()}")

# 2. queue.Queue (thread-safe, for multi-threading)
q2 = queue.Queue()
q2.put(1)
q2.put(2)
print(f"queue.Queue: get: {q2.get()}")

# 3. queue.LifoQueue (thread-safe stack)
q3 = queue.LifoQueue()
q3.put(1)
q3.put(2)
print(f"LifoQueue (stack): get: {q3.get()}")

# 4. queue.PriorityQueue (thread-safe priority queue)
q4 = queue.PriorityQueue()
q4.put((2, "medium"))
q4.put((1, "high"))
q4.put((3, "low"))
print(f"PriorityQueue: get: {q4.get()}")

print("\nUse deque for most cases, queue.* for thread-safety")

***Figure 5.17:** Python offers multiple queue implementations for different use cases.*

---
## 6. Queue Implementations


### Implementation 1: Linked List-Based Queue

**Listing 5.13 — Linked List Queue**

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedQueue:
    """Queue using singly linked list with head and tail."""
    
    def __init__(self):
        self._head = None  # Front of queue
        self._tail = None  # Rear of queue
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def __str__(self):
        items = []
        current = self._head
        while current:
            items.append(str(current.data))
            current = current.next
        return f"Queue[{' <- '.join(items)}]"
    
    def is_empty(self):
        return self._head is None
    
    def enqueue(self, item):
        """Add to rear - O(1)."""
        new_node = Node(item)
        if self.is_empty():
            self._head = self._tail = new_node
        else:
            self._tail.next = new_node
            self._tail = new_node
        self._size += 1
    
    def dequeue(self):
        """Remove from front - O(1)."""
        if self.is_empty():
            raise IndexError("Dequeue from empty queue")
        data = self._head.data
        self._head = self._head.next
        if self._head is None:
            self._tail = None
        self._size -= 1
        return data
    
    def front(self):
        """Peek at front - O(1)."""
        if self.is_empty():
            raise IndexError("Front of empty queue")
        return self._head.data

# Test
queue = LinkedQueue()
for x in ['A', 'B', 'C', 'D']:
    queue.enqueue(x)
    print(f"Enqueue '{x}': {queue}")

print()
while not queue.is_empty():
    print(f"Dequeue: {queue.dequeue()}, Remaining: {queue}")

***Figure 5.13:** Linked queue adds at tail, removes from head—both O(1) with two pointers.*

### Implementation 2: Circular Array Queue

**Listing 5.14 — Circular Array Queue**

In [ ]:
class CircularQueue:
    """Fixed-size queue using circular array."""
    
    def __init__(self, capacity):
        self._data = [None] * capacity
        self._capacity = capacity
        self._front = 0
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def is_full(self):
        return self._size == self._capacity
    
    def enqueue(self, item):
        """Add to rear - O(1)."""
        if self.is_full():
            raise OverflowError("Queue is full")
        rear = (self._front + self._size) % self._capacity
        self._data[rear] = item
        self._size += 1
    
    def dequeue(self):
        """Remove from front - O(1)."""
        if self.is_empty():
            raise IndexError("Dequeue from empty queue")
        item = self._data[self._front]
        self._data[self._front] = None
        self._front = (self._front + 1) % self._capacity
        self._size -= 1
        return item
    
    def front(self):
        if self.is_empty():
            raise IndexError("Front of empty queue")
        return self._data[self._front]
    
    def display(self):
        """Show internal array state."""
        markers = ['F' if i == self._front else ' ' for i in range(self._capacity)]
        rear = (self._front + self._size - 1) % self._capacity if self._size > 0 else -1
        if rear >= 0:
            markers[rear] = 'R' if markers[rear] == ' ' else 'FR'
        return f"Array: {self._data}, Front={self._front}, Size={self._size}"

# Test
q = CircularQueue(5)
print("Circular Queue Demo (capacity=5):\n")

for x in [1, 2, 3]:
    q.enqueue(x)
    print(f"Enqueue {x}: {q.display()}")

print(f"\nDequeue: {q.dequeue()}")
print(f"Dequeue: {q.dequeue()}")
print(f"{q.display()}")

for x in [4, 5, 6]:
    q.enqueue(x)
    print(f"Enqueue {x}: {q.display()}")

print("\nNotice how elements wrap around the circular array!")

***Figure 5.14:** Circular array queue wraps around using modulo arithmetic. No element shifting needed.*

---
## 7. Queue Variants


### Double-Ended Queue (Deque)

**Listing 5.15 — Deque Operations**

In [ ]:
from collections import deque

# Deque allows O(1) operations at BOTH ends
d = deque()

# Add to both ends
d.append(2)      # Add to right: [2]
d.append(3)      # Add to right: [2, 3]
d.appendleft(1)  # Add to left:  [1, 2, 3]
d.appendleft(0)  # Add to left:  [0, 1, 2, 3]

print(f"After additions: {list(d)}")

# Remove from both ends
right = d.pop()       # Remove from right
left = d.popleft()    # Remove from left
print(f"Popped right: {right}, left: {left}")
print(f"Remaining: {list(d)}")

# Rotation
d = deque([1, 2, 3, 4, 5])
print(f"\nOriginal: {list(d)}")

d.rotate(2)  # Rotate right by 2
print(f"Rotate right 2: {list(d)}")

d.rotate(-3)  # Rotate left by 3
print(f"Rotate left 3: {list(d)}")

# Max length deque (sliding window)
print("\nMax-length deque (maxlen=3):")
bounded = deque(maxlen=3)
for x in range(6):
    bounded.append(x)
    print(f"  Append {x}: {list(bounded)}")

***Figure 5.15:** Deque supports both stack and queue operations. The maxlen parameter creates a sliding window.*

### Priority Queue

**Listing 5.16 — Priority Queue with heapq**

In [ ]:
import heapq

# heapq implements a min-heap (smallest first)
pq = []

# Add items with priorities
heapq.heappush(pq, (3, "Medium priority"))
heapq.heappush(pq, (1, "High priority"))
heapq.heappush(pq, (5, "Low priority"))
heapq.heappush(pq, (2, "Higher priority"))

print("Priority Queue (min-heap):")
print(f"  Heap: {pq}\n")

# Remove in priority order (lowest first)
print("Removing in priority order:")
while pq:
    priority, task = heapq.heappop(pq)
    print(f"  Priority {priority}: {task}")

# For max-heap, negate priorities
print("\nMax-Heap simulation (negate priorities):")
max_pq = []
for item in [(3, "C"), (1, "A"), (5, "E"), (2, "B")]:
    heapq.heappush(max_pq, (-item[0], item[1]))

while max_pq:
    priority, value = heapq.heappop(max_pq)
    print(f"  Priority {-priority}: {value}")

***Figure 5.16:** Priority queue serves highest-priority items first. Python's heapq is a min-heap by default.*

### Monotonic Queue

**Listing 5.17 — Sliding Window Maximum**

In [ ]:
from collections import deque

def sliding_window_max(nums, k):
    """
    Find maximum in each sliding window of size k.
    Uses monotonic decreasing deque.
    Time: O(n), Space: O(k)
    """
    result = []
    dq = deque()  # Stores indices, values in decreasing order
    
    for i, num in enumerate(nums):
        # Remove indices outside current window
        while dq and dq[0] <= i - k:
            dq.popleft()
        
        # Remove smaller elements (they can't be max)
        while dq and nums[dq[-1]] < num:
            dq.pop()
        
        dq.append(i)
        
        # Add to result once we have full window
        if i >= k - 1:
            result.append(nums[dq[0]])
    
    return result

# Test
nums = [1, 3, -1, -3, 5, 3, 6, 7]
k = 3

print(f"Array: {nums}")
print(f"Window size: {k}")
print(f"\nSliding windows and their max:")

for i in range(len(nums) - k + 1):
    window = nums[i:i+k]
    print(f"  {window} -> max = {max(window)}")

print(f"\nResult: {sliding_window_max(nums, k)}")

***Figure 5.17:** Monotonic deque maintains candidates for window maximum in decreasing order.*

---
## 8. Queue Applications


### Application 1: Breadth-First Search (BFS)

**Listing 5.18 — BFS with Queue**

In [ ]:
from collections import deque

def bfs(graph, start):
    """
    Breadth-First Search using a queue.
    Visits nodes level by level.
    """
    visited = set()
    queue = deque([start])
    visited.add(start)
    order = []
    
    while queue:
        node = queue.popleft()
        order.append(node)
        
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    
    return order

# Example graph
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("Graph:")
for node, neighbors in graph.items():
    print(f"  {node} -> {neighbors}")

print(f"\nBFS from 'A': {bfs(graph, 'A')}")
print("(Visits all nodes at distance 1 before distance 2, etc.)")

***Figure 5.18:** BFS uses a queue to explore nodes level by level—essential for shortest path in unweighted graphs.*

### Application 2: Level Order Tree Traversal

**Listing 5.19 — Level Order Traversal**

In [ ]:
from collections import deque

class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

def level_order(root):
    """
    Return list of lists, each containing values at that level.
    """
    if not root:
        return []
    
    result = []
    queue = deque([root])
    
    while queue:
        level_size = len(queue)
        level_values = []
        
        for _ in range(level_size):
            node = queue.popleft()
            level_values.append(node.val)
            
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
        
        result.append(level_values)
    
    return result

# Build tree:
#        1
#       / \
#      2   3
#     / \   \
#    4   5   6
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)
root.right.right = TreeNode(6)

print("Tree structure:")
print("       1")
print("      / \\")
print("     2   3")
print("    / \\   \\")
print("   4   5   6")

print(f"\nLevel order traversal: {level_order(root)}")

***Figure 5.19:** Level order traversal visits tree nodes layer by layer using a queue.*

### Application 3: Task Scheduler

**Listing 5.20 — Simple Task Queue**

In [ ]:
from collections import deque
import time

class TaskScheduler:
    """Simple FIFO task scheduler."""
    
    def __init__(self):
        self.queue = deque()
    
    def add_task(self, task_name, task_func):
        """Add task to queue."""
        self.queue.append((task_name, task_func))
        print(f"Added task: {task_name}")
    
    def run_all(self):
        """Execute all tasks in FIFO order."""
        print("\nExecuting tasks:")
        while self.queue:
            name, func = self.queue.popleft()
            print(f"  Running: {name}")
            func()
        print("All tasks completed!")

# Demo
scheduler = TaskScheduler()

scheduler.add_task("Download data", lambda: print("    → Data downloaded"))
scheduler.add_task("Process data", lambda: print("    → Data processed"))
scheduler.add_task("Generate report", lambda: print("    → Report generated"))
scheduler.add_task("Send email", lambda: print("    → Email sent"))

scheduler.run_all()

***Figure 5.26:** Task schedulers use queues to process jobs in the order they arrive.*

### Application 4: Shortest Path in Binary Matrix

**Listing 5.27 — BFS Shortest Path**

In [ ]:
from collections import deque

def shortest_path_binary_matrix(grid):
    """
    Find shortest path from top-left to bottom-right.
    Can move in 8 directions. 0 = open, 1 = blocked.
    """
    n = len(grid)
    if grid[0][0] == 1 or grid[n-1][n-1] == 1:
        return -1
    
    # BFS with queue: (row, col, distance)
    queue = deque([(0, 0, 1)])
    grid[0][0] = 1  # Mark visited
    
    directions = [(-1,-1),(-1,0),(-1,1),(0,-1),
                  (0,1),(1,-1),(1,0),(1,1)]
    
    while queue:
        row, col, dist = queue.popleft()
        
        if row == n-1 and col == n-1:
            return dist
        
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            if (0 <= new_row < n and 0 <= new_col < n and
                grid[new_row][new_col] == 0):
                grid[new_row][new_col] = 1  # Mark visited
                queue.append((new_row, new_col, dist + 1))
    
    return -1

# Test
grid = [
    [0, 0, 0],
    [1, 1, 0],
    [1, 1, 0]
]

print("Grid (0=open, 1=blocked):")
for row in grid:
    print(f"  {row}")

# Make a copy since function modifies grid
import copy
grid_copy = copy.deepcopy(grid)
result = shortest_path_binary_matrix(grid_copy)
print(f"\nShortest path length: {result}")

***Figure 5.27:** BFS naturally finds shortest paths in unweighted graphs/grids.*

### Application 5: Generate Binary Numbers

**Listing 5.28 — Generate Binary Numbers 1 to N**

In [ ]:
from collections import deque

def generate_binary_numbers(n):
    """
    Generate binary representations of numbers 1 to n.
    Uses queue to build each binary number from previous.
    """
    result = []
    queue = deque(['1'])
    
    for _ in range(n):
        # Get front of queue
        current = queue.popleft()
        result.append(current)
        
        # Generate next two binary numbers
        queue.append(current + '0')
        queue.append(current + '1')
    
    return result

# Test
n = 10
binaries = generate_binary_numbers(n)

print(f"Binary numbers 1 to {n}:")
for i, b in enumerate(binaries, 1):
    print(f"  {i:2d} = {b}")

***Figure 5.28:** Queue generates binary numbers by appending '0' and '1' to each dequeued value.*

### Application 6: Hot Potato Game

**Listing 5.21 — Hot Potato Simulation**

In [ ]:
from collections import deque

def hot_potato(names, num_passes):
    """
    Simulate hot potato game.
    Pass potato around; every num_passes, that person is out.
    """
    queue = deque(names)
    
    print(f"Starting with: {list(queue)}")
    print(f"Passing {num_passes} times before elimination\n")
    
    round_num = 1
    while len(queue) > 1:
        # Pass the potato num_passes times
        for _ in range(num_passes):
            queue.append(queue.popleft())  # Move front to back
        
        eliminated = queue.popleft()
        print(f"Round {round_num}: {eliminated} is eliminated")
        print(f"  Remaining: {list(queue)}")
        round_num += 1
    
    winner = queue[0]
    print(f"\n🏆 Winner: {winner}")
    return winner

# Play the game
players = ["Alice", "Bob", "Charlie", "Diana", "Eve"]
hot_potato(players, 7)

***Figure 5.21:** The hot potato game is a circular process naturally modeled by rotating a queue.*

---
## 9. Stacks vs Queues


### Side-by-Side Comparison

| Aspect | Stack | Queue |
| --- | --- | --- |
| Principle | LIFO (Last In, First Out) | FIFO (First In, First Out) |
| Add operation | push() - to top | enqueue() - to rear |
| Remove operation | pop() - from top | dequeue() - from front |
| Python built-in | list (append/pop) | collections.deque |
| Use cases | Undo, parsing, DFS, recursion | BFS, scheduling, buffering |
| Access pattern | Most recent first | Oldest first |

**Listing 5.22 — Stack for DFS vs Queue for BFS**

In [ ]:
from collections import deque

def dfs_iterative(graph, start):
    """DFS using a stack - goes deep first."""
    visited = set()
    stack = [start]
    order = []
    
    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            order.append(node)
            # Add neighbors (reversed to maintain left-to-right order)
            for neighbor in reversed(graph.get(node, [])):
                if neighbor not in visited:
                    stack.append(neighbor)
    return order

def bfs(graph, start):
    """BFS using a queue - goes wide first."""
    visited = set()
    queue = deque([start])
    visited.add(start)
    order = []
    
    while queue:
        node = queue.popleft()
        order.append(node)
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    return order

# Tree-like graph:
#        A
#       /|\
#      B C D
#     /|   |
#    E F   G
graph = {
    'A': ['B', 'C', 'D'],
    'B': ['E', 'F'],
    'C': [],
    'D': ['G'],
    'E': [], 'F': [], 'G': []
}

print("Graph (tree structure):")
print("       A")
print("      /|\\")
print("     B C D")
print("    /|   |")
print("   E F   G")
print()
print(f"DFS (stack): {dfs_iterative(graph, 'A')}")
print(f"BFS (queue): {bfs(graph, 'A')}")

***Figure 5.22:** DFS with stack explores deeply; BFS with queue explores broadly. Same code structure, different data structure!*

### Implementing Queue with Stacks

**Listing 5.23 — Queue Using Two Stacks**

In [ ]:
class QueueUsingStacks:
    """
    Implement queue using two stacks.
    Push: O(1), Pop: O(1) amortized
    """
    
    def __init__(self):
        self.inbox = []   # For enqueue
        self.outbox = []  # For dequeue
    
    def enqueue(self, x):
        """Add to inbox stack."""
        self.inbox.append(x)
    
    def dequeue(self):
        """Transfer to outbox if empty, then pop."""
        if not self.outbox:
            while self.inbox:
                self.outbox.append(self.inbox.pop())
        if not self.outbox:
            raise IndexError("Dequeue from empty queue")
        return self.outbox.pop()
    
    def peek(self):
        if not self.outbox:
            while self.inbox:
                self.outbox.append(self.inbox.pop())
        if not self.outbox:
            raise IndexError("Peek at empty queue")
        return self.outbox[-1]

# Demo
q = QueueUsingStacks()
print("Queue using two stacks:")

for x in [1, 2, 3]:
    q.enqueue(x)
    print(f"  Enqueue {x}: inbox={q.inbox}, outbox={q.outbox}")

print(f"\n  Dequeue: {q.dequeue()}")
print(f"  After: inbox={q.inbox}, outbox={q.outbox}")

q.enqueue(4)
print(f"\n  Enqueue 4: inbox={q.inbox}, outbox={q.outbox}")

print(f"\n  Dequeue: {q.dequeue()}")
print(f"  Dequeue: {q.dequeue()}")
print(f"  Dequeue: {q.dequeue()}")

***Figure 5.23:** Classic interview problem: queue from two stacks. Elements reverse when transferred.*

### Implementing Stack with Queues

**Listing 5.24 — Stack Using Two Queues**

In [ ]:
from collections import deque

class StackUsingQueues:
    """
    Implement stack using two queues.
    Push: O(n), Pop: O(1)
    """
    
    def __init__(self):
        self.q1 = deque()
        self.q2 = deque()
    
    def push(self, x):
        """Add to q2, transfer all from q1 to q2, swap."""
        self.q2.append(x)
        while self.q1:
            self.q2.append(self.q1.popleft())
        self.q1, self.q2 = self.q2, self.q1
    
    def pop(self):
        """Pop from q1."""
        if not self.q1:
            raise IndexError("Pop from empty stack")
        return self.q1.popleft()
    
    def top(self):
        if not self.q1:
            raise IndexError("Top of empty stack")
        return self.q1[0]

# Demo
s = StackUsingQueues()
print("Stack using two queues:")

for x in [1, 2, 3]:
    s.push(x)
    print(f"  Push {x}: q1={list(s.q1)}")

print(f"\n  Pop: {s.pop()}")
print(f"  Pop: {s.pop()}")
print(f"  After pops: q1={list(s.q1)}")

***Figure 5.24:** Stack from two queues: each push rearranges so newest is always at front.*

- **Stack (LIFO):** push/pop at one end; use for undo, parsing, DFS, recursion
- **Queue (FIFO):** enqueue at rear, dequeue at front; use for BFS, scheduling, buffering
- **Python:** Use `list` for stacks, `collections.deque` for queues
- **All operations:** O(1) when implemented correctly
- **Deque:** Supports both stack and queue operations
- **Priority Queue:** Removes highest-priority first (use `heapq`)

---
## 10. Common Pitfalls


### Pitfall 1: Using List as Queue

**Listing 5.25 — List vs Deque for Queue**

In [ ]:
import time
from collections import deque

n = 10000

# BAD: Using list as queue
bad_queue = []
start = time.perf_counter()
for i in range(n):
    bad_queue.append(i)
for i in range(n):
    bad_queue.pop(0)  # O(n) each time!
bad_time = time.perf_counter() - start

# GOOD: Using deque as queue
good_queue = deque()
start = time.perf_counter()
for i in range(n):
    good_queue.append(i)
for i in range(n):
    good_queue.popleft()  # O(1)
good_time = time.perf_counter() - start

print(f"Queue operations ({n} enqueue + {n} dequeue):")
print(f"  list.pop(0):      {bad_time*1000:.2f} ms")
print(f"  deque.popleft():  {good_time*1000:.2f} ms")
speedup = bad_time/good_time if good_time > 0 else float('inf')
print(f"  Speedup: {speedup:.1f}x")
print(f"\nAlways use deque for queues!")

***Figure 5.25:** `list.pop(0)` is O(n) because it shifts all elements. Use `deque.popleft()` for O(1).*

### Pitfall 2: Stack Overflow in Recursion

**Listing 5.26 — Recursion vs Explicit Stack**

In [ ]:
import sys

print(f"Default recursion limit: {sys.getrecursionlimit()}")

# Recursive factorial - limited by call stack
def factorial_recursive(n):
    if n <= 1:
        return 1
    return n * factorial_recursive(n - 1)

# Iterative with explicit stack - no limit
def factorial_iterative(n):
    stack = list(range(1, n + 1))
    result = 1
    while stack:
        result *= stack.pop()
    return result

# Test
print(f"\nFactorial of 10:")
print(f"  Recursive: {factorial_recursive(10)}")
print(f"  Iterative: {factorial_iterative(10)}")

# Deep recursion example
def deep_recursion(n):
    if n == 0:
        return 0
    return deep_recursion(n - 1)

# This would cause RecursionError for large n
try:
    deep_recursion(500)
    print("\ndeep_recursion(500): Success")
except RecursionError:
    print("\ndeep_recursion(500): RecursionError!")

print("\nTip: Convert deep recursion to iteration with explicit stack")

***Figure 5.26:** Python's recursion limit can cause RecursionError. Use an explicit stack for deep operations.*

### Pitfall 3: Empty Stack/Queue Operations

**Listing 5.27 — Safe Operations**

In [ ]:
from collections import deque

# BAD: Not checking empty before pop
def bad_stack_usage():
    stack = []
    try:
        stack.pop()  # IndexError!
    except IndexError as e:
        print(f"Error: {e}")

# GOOD: Check before operating
def good_stack_usage():
    stack = []
    if stack:  # Check if not empty
        return stack.pop()
    else:
        return None  # Or handle gracefully

# GOOD: Use try-except
def safe_pop(stack):
    try:
        return stack.pop()
    except IndexError:
        return None

print("Empty stack operations:")
bad_stack_usage()

stack = []
print(f"safe_pop on empty: {safe_pop(stack)}")

stack = [1, 2, 3]
print(f"safe_pop on [1,2,3]: {safe_pop(stack)}")

***Figure 5.27:** Always check if stack/queue is empty before popping, or use try-except.*

---
# 📝 Exercises


### Exercise 1: Min Stack  (⭐⭐ Medium)

Design a stack that supports push, pop, top, and retrieving the minimum element in O(1) time.

**Expected:** (Expected: All operations should work in constant time)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Auxiliary Stack:**

                        Maintain a second stack `min_stack` to track minimum values.
- **Hint 2 - Push Logic:**

                        Push to min_stack only when new value <= current min (or stack is empty).
- **Hint 3 - Pop Logic:**

                        Pop from min_stack when the popped value equals the current minimum.
</details>

In [ ]:
# ✏️ [EX1]
# Min Stack - O(1) push, pop, top, and getMin

class MinStack:
    def __init__(self):
        # Your code here
        pass
    
    def push(self, val):
        # Your code here
        pass
    
    def pop(self):
        # Your code here
        pass
    
    def top(self):
        # Your code here
        pass
    
    def get_min(self):
        # Your code here
        pass

# Test your implementation (uncomment)
# ms = MinStack()
# ms.push(-2)
# ms.push(0)
# ms.push(-3)
# print(f"getMin: {ms.get_min()}")  # -3
# ms.pop()
# print(f"top: {ms.top()}")  # 0
# print(f"getMin: {ms.get_min()}")  # -2

### Exercise 2: Valid Parentheses with Multiple Types  (⭐⭐ Medium)

Extend the balanced parentheses checker to also verify that brackets are properly nested (e.g., `[{()}]` is valid but `[{(})]` is not).

**Expected:** (Expected: Proper nesting of (), [], {} brackets)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Create Mapping:**

                        Map closing brackets to opening: `{')': '(', ']': '[', '}': '{'}`
- **Hint 2 - Push Opening:**

                        When you see an opening bracket, push it onto the stack.
- **Hint 3 - Match Closing:**

                        For closing brackets, check if stack is non-empty and top matches the expected opening bracket.
</details>

In [ ]:
# ✏️ [EX2]
# Valid Parentheses - Check proper nesting of (), [], {}

def is_valid_nesting(s):
    # Your code here
    pass

# Test your implementation (uncomment)
# test_cases = ["[{()}]", "[{(})]", "((()))", "([)]", ""]
# for s in test_cases:
#     result = "✓" if is_valid_nesting(s) else "✗"
#     print(f"{result} '{s}'")

### Exercise 3: Implement Circular Queue  (⭐⭐ Medium)

Implement a circular queue with operations: enqueue, dequeue, front, rear, isEmpty, isFull.

**Expected:** (Expected: Use modulo arithmetic for wrapping)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Track Position:**

                        Use front index and size (or front and rear pointers).
- **Hint 2 - Modulo Wrapping:**

                        Calculate rear position as `(front + size) % capacity`
- **Hint 3 - Full vs Empty:**

                        Full when `size == capacity`, empty when `size == 0`
</details>

In [ ]:
# ✏️ [EX3]
# Circular Queue - Fixed size queue with wrapping

class MyCircularQueue:
    def __init__(self, k):
        # Your code here
        pass
    
    def enqueue(self, value):
        # Your code here
        pass
    
    def dequeue(self):
        # Your code here
        pass
    
    def front(self):
        # Your code here
        pass
    
    def rear(self):
        # Your code here
        pass

# Test your implementation (uncomment)
# cq = MyCircularQueue(3)
# print(cq.enqueue(1))  # True
# print(cq.enqueue(2))  # True
# print(cq.enqueue(3))  # True
# print(cq.enqueue(4))  # False (full)
# print(cq.rear())      # 3
# print(cq.dequeue())   # True
# print(cq.enqueue(4))  # True
# print(cq.rear())      # 4

### Exercise 4: Daily Temperatures  (⭐⭐⭐ Hard)

Given daily temperatures, return an array where `answer[i]` is the number of days until a warmer temperature. Use a monotonic stack.

**Expected:** (Expected: [73,74,75,71,69,72,76,73] → [1,1,4,2,1,1,0,0])

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Store Indices:**

                        Use a stack to store indices of temperatures (not the temperatures themselves).
- **Hint 2 - Monotonic Decreasing:**

                        Maintain a decreasing stack. Pop when current temp > temp at stack top index.
- **Hint 3 - Calculate Days:**

                        When popping, the answer for that index is `current_index - popped_index`
</details>

In [ ]:
# ✏️ [EX4]
# Daily Temperatures - Days until warmer temperature

def daily_temperatures(temperatures):
    # Your code here
    pass

# Test your implementation (uncomment)
# temps = [73, 74, 75, 71, 69, 72, 76, 73]
# print(f"Temperatures: {temps}")
# print(f"Days to wait: {daily_temperatures(temps)}")
# Expected: [1, 1, 4, 2, 1, 1, 0, 0]

### Exercise 5: Number of Recent Calls  (⭐ Easy)

Implement a RecentCounter class that counts the number of calls made in the last 3000 milliseconds.

**Expected:** (Expected: Use a queue and remove old timestamps)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Use a Deque:**

                        Use `collections.deque` to store timestamps efficiently.
- **Hint 2 - Add and Clean:**

                        On each ping, add the new timestamp, then remove old ones.
- **Hint 3 - Time Window:**

                        Remove timestamps that are older than `t - 3000` (outside the window).
</details>

In [ ]:
# ✏️ [EX5]
# Number of Recent Calls - Count calls in last 3000ms

from collections import deque

class RecentCounter:
    def __init__(self):
        # Your code here
        pass
    
    def ping(self, t):
        """
        Add new request at time t.
        Return number of requests in [t-3000, t].
        """
        # Your code here
        pass

# Test your implementation (uncomment)
# rc = RecentCounter()
# print(rc.ping(1))     # 1 (requests in [0, 1])
# print(rc.ping(100))   # 2 (requests in [0, 100])
# print(rc.ping(3001))  # 3 (requests in [1, 3001])
# print(rc.ping(3002))  # 3 (requests in [2, 3002], 1 is out)

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,12}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-12 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
_ORIGINAL_HASHES = {}

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request
import time as _time, datetime as _dt

WEEK = "Week_05"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ══════════════════════════════════════════════════════════
# SELF-HEALING TIMER — works even if Cell 1 was skipped
# ══════════════════════════════════════════════════════════
_TRACKER_FILE = os.path.join(
    '/content' if os.path.isdir('/content') else '/tmp',
    f'.cp2_session_{WEEK}.json'
)
IDLE_THRESHOLD = 300

def _sh_calc_active(heartbeats):
    """Calculate active seconds from a heartbeat list."""
    if len(heartbeats) < 2: return 0
    active = 0
    for i in range(1, len(heartbeats)):
        gap = heartbeats[i] - heartbeats[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _collect_timing():
    """
    Collect timing data from all available sources:
      1. In-memory variables (timer cell ran in this kernel)
      2. Persistent tracker file (previous sessions / kernel restarts)
      3. Minimal fallback (nothing available)
    """
    _info = {
        'active_time': 0, 'wall_time': 0, 'cells_run': 0,
        'heartbeat_count': 0, 'session_count': 0,
        'session_start': '', 'session_end': _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'none'
    }

    # ── Source 1: In-memory (timer cell ran this kernel) ──
    _has_memory = False
    try:
        _s = _SESSION_START        # noqa: F821
        _hb = list(_HEARTBEATS)    # noqa: F821
        _cr = _CELLS_RUN[0]        # noqa: F821
        _has_memory = True
        _info['active_time'] = _sh_calc_active(_hb)
        _info['wall_time'] = int(_time.time() - _s)
        _info['cells_run'] = _cr
        _info['heartbeat_count'] = len(_hb)
        _info['session_start'] = _SESSION_START_STR  # noqa: F821
        _info['source'] = 'memory'
    except NameError:
        pass

    # ── Source 2: Persistent file (adds previous sessions) ──
    try:
        with open(_TRACKER_FILE, 'r') as f:
            _t = json.load(f)
        sessions = _t.get('sessions', [])
        _info['session_count'] = len(sessions)

        if not _has_memory and sessions:
            # Timer cell was NOT run — reconstruct from file
            last = sessions[-1]
            hb = last.get('heartbeats', [])
            _info['active_time'] = _sh_calc_active(hb)
            _info['wall_time'] = int(hb[-1] - hb[0]) if len(hb) >= 2 else 0
            _info['cells_run'] = last.get('cells_run', len(hb))
            _info['heartbeat_count'] = len(hb)
            _info['session_start'] = last.get('start', '')
            _info['source'] = 'file-current'

        # Add previous sessions' active time
        if len(sessions) > 1:
            prev_active = 0
            prev_cells = 0
            prev_hb = 0
            cutoff = -1 if _has_memory else -1  # exclude current session
            for sess in sessions[:cutoff]:
                hb = sess.get('heartbeats', [])
                prev_active += _sh_calc_active(hb)
                prev_cells += sess.get('cells_run', len(hb))
                prev_hb += len(hb)
            _info['prev_active_time'] = prev_active
            _info['prev_cells_run'] = prev_cells
            _info['prev_heartbeats'] = prev_hb
            _info['total_active_time'] = _info['active_time'] + prev_active
            _info['total_cells_run'] = _info['cells_run'] + prev_cells
            if _info['source'] == 'none':
                _info['source'] = 'file-prev'

    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError):
        pass

    # Ensure totals exist
    _info.setdefault('total_active_time', _info['active_time'])
    _info.setdefault('total_cells_run', _info['cells_run'])
    return _info

# ── Persist final heartbeat snapshot before collecting ──
try:
    _t = json.load(open(_TRACKER_FILE))
    if _t['sessions']:
        _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
        _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
        json.dump(_t, open(_TRACKER_FILE, 'w'))
except Exception:
    pass

_timing = _collect_timing()

# ── Display timing ──
if _timing['source'] == 'none':
    print("⚠️  Zamanlayıcı verisi bulunamadı — ilk hücreyi çalıştırmayı unutmayın.")
    print("    Gönderim yine de yapılacak (süre: 0 olarak kaydedilir).")
else:
    _a = _timing['active_time']
    _w = _timing['wall_time']
    print(f"⏱️  Bu oturum — Aktif: {_a//60}m {_a%60}s  |  Duvar: {_w//60}m {_w%60}s")
    print(f"🔢  Hücreler: {_timing['cells_run']}  |  Heartbeat: {_timing['heartbeat_count']}")
    if _timing['session_count'] > 1:
        _ta = _timing['total_active_time']
        print(f"📂  Toplam {_timing['session_count']} oturum — Toplam aktif: {_ta//60}m {_ta%60}s  |  Toplam hücre: {_timing['total_cells_run']}")
    if _timing['total_active_time'] < 120:
        print("⚠️  Toplam aktif süre < 2 dk — çalıştırdığınız hücre sayısı düşük olabilir.")

_time_on_page = _timing['total_active_time']

# ══════════════════════════════════════════════════════════
# STUDENT INFO CHECK
# ══════════════════════════════════════════════════════════
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın (öğrenci bilgileri).")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın — bazı alanlar boş.")

# ══════════════════════════════════════════════════════════
# EXERCISE ANSWER COLLECTION (improved with retry)
# ══════════════════════════════════════════════════════════
_EX_PATTERN = re.compile(r'#\s*✏️\s*\[EX(\d+)\]')

def _extract_exercise(src):
    """Try to extract exercise ID and code from a source string."""
    m = _EX_PATTERN.match(src)
    if not m: return None, None
    ex_id = 'ex' + m.group(1)
    clean = '\n'.join(src.split('\n')[1:]).strip()
    return ex_id, clean

_answers = {}

# ── Method 1: In[] list (most reliable for Run-All) ──
try:
    for _src in In:
        if not _src: continue
        _eid, _code = _extract_exercise(_src)
        if _eid:
            _answers[_eid] = {
                'code': _code,
                'modified': len(_code) > 5
            }
except NameError:
    pass

# ── Method 2: IPython history_manager (flush first!) ──
if not _answers:
    try:
        _ipy = get_ipython()
        # Force flush so Run-All history is fully written
        try: _ipy.history_manager.db.commit()
        except Exception: pass
        _time.sleep(0.5)  # small delay for DB sync
        for _sess, _line, _src in _ipy.history_manager.get_range(output=False):
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': len(_code) > 5
                }
    except Exception:
        pass

# ── Method 3: Read notebook file (VS Code / local Jupyter) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir('.') if f.endswith('.ipynb') and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), 'r', encoding='utf-8') as _f:
            _nb = json.load(_f)
        for _cell in _nb['cells']:
            if _cell['cell_type'] != 'code': continue
            _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': len(_code) > 5
                }

# ── Method 4: Colab notebook file at /content/ ──
if not _answers:
    try:
        _colab_candidates = [f for f in os.listdir('/content') if f.endswith('.ipynb') and WEEK in f]
        if _colab_candidates:
            _nb_path = os.path.join('/content', _colab_candidates[0])
            with open(_nb_path, 'r', encoding='utf-8') as _f:
                _nb = json.load(_f)
            for _cell in _nb['cells']:
                if _cell['cell_type'] != 'code': continue
                _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
                _eid, _code = _extract_exercise(_src)
                if _eid:
                    _answers[_eid] = {
                        'code': _code,
                        'modified': len(_code) > 5
                    }
    except Exception:
        pass

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys(), key=lambda x: int(x[2:]) if x[2:].isdigit() else 999))}")

if not _answers:
    print("\n⚠️  Hiçbir exercise yanıtı bulunamadı!")
    print("Submit'ten ÖNCE tüm exercise hücrelerini çalıştırdığınızdan emin olun.")
    print("\n💡 İpucu: 'Run All' yerine exercise hücrelerini tek tek çalıştırıp")
    print("   ardından bu hücreyi çalıştırmayı deneyin.")
    raise SystemExit()

# ══════════════════════════════════════════════════════════
# SEND SUBMISSION
# ══════════════════════════════════════════════════════════
_data = json.dumps({
    'week': WEEK,
    'studentId': _sid,
    'studentName': _sname,
    'studentEmail': _semail,
    'classCode': _scode,
    'source': 'dsa-notebook',
    'timeOnPage': _time_on_page,
    'wallTime': _timing['wall_time'],
    'cellsRun': _timing['total_cells_run'],
    'sessionCount': _timing['session_count'],
    'sessionStart': _timing['session_start'],
    'sessionEnd': _timing['session_end'],
    'timerSource': _timing['source'],
    'answers': _answers
}).encode('utf-8')

print("📡 Gönderiliyor...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get('success'):
        print(f"\n✅ {_result['message']}")
        print('📧 Onay için e-postanızı kontrol edin.')
    else:
        print(f"\n❌ {_result.get('message', 'Gönderim başarısız')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  İstek gönderildi — onay için e-postanızı kontrol edin.")
    print(f"(E-posta gelmezse tekrar deneyin veya hocaya başvurun)")
